# Creating an Implementation Object

An `Implementation` object represents an instance of a synthetic biological construct. It primarily describes the **build phase** of a design-build-test-learn workflow. An `Implementation` can refer to a physical laboratory sample that has already been built, or one that is planned to be built in the future. It can also represent virtual or simulated instances.

Key properties of an `Implementation`:

- `wasDerivedFrom`: Inherited from the `Identified` superclass, this property is used to link the `Implementation` back to its original design, which can be either a `ModuleDefinition` or a `ComponentDefinition`.

- `built`: This property is OPTIONAL and links to a `TopLevel` object (either a `ComponentDefinition` or `ModuleDefinition`) that describes the *actual physical structure and/or functional behavior* of the `Implementation`.
    - If `built` refers to the **same** `ComponentDefinition`/`ModuleDefinition` as `wasDerivedFrom`, it implies that the actual structure/function of the `Implementation` matches its original design.
    - If `built` refers to a **different** `ComponentDefinition`/`ModuleDefinition`, it indicates that the `Implementation` has deviated from the original design. This is useful for documenting scenarios like DNA sequencing results not matching the target sequence after assembly.

This tutorial will demonstrate how to create an `Implementation` that links to an original design and then to a *mutated* or *deviated* actual built construct, illustrating the use of both `wasDerivedFrom` and `built` properties.

For more details, refer to Section 7.12 "Implementation" on page 52 in the SBOL specification (e.g., SBOL 2.3.0 or later).

In [1]:
import sbol2

### Document Setup
First, we'll create an SBOL Document and set a homespace. The homespace defines the base URI for objects created in this document, helping to ensure unique identifiers.

In [2]:
doc = sbol2.Document()
sbol2.setHomespace('http://examples.org/ImplementationNotebook')

### Define the Original Design
We'll start by defining a `ComponentDefinition` that represents our **original design** for a gene, for example, a GFP coding sequence.

In [3]:
# Create a Sequence for the original GFP design
gfp_design_sequence = sbol2.Sequence('gfp_design_sequence')
gfp_design_sequence.elements = 'atggactacaaagacgatgacgacaaagctcttttagatccagtaatgaccttcac'
gfp_design_sequence.encoding = sbol2.SBOL_ENCODING_IUPAC
doc.addSequence(gfp_design_sequence)

# Create the ComponentDefinition for the original GFP design
gfp_design = sbol2.ComponentDefinition('gfp_design')
gfp_design.types =  [sbol2.BIOPAX_DNA]
gfp_design.roles = 'http://identifiers.org/so/SO:0000316' # Role: coding sequence (CDS)
gfp_design.displayId = 'gfp_design'
gfp_design.version = '1.0.0'
gfp_design.sequences = [gfp_design_sequence]

doc.addComponentDefinition(gfp_design)

### Define the Actual Built Construct (with deviation)
Next, we'll define another `ComponentDefinition` that represents the **actual construct that was built**, which has a slight deviation (e.g., a mutation detected by sequencing) from the original design.

In [4]:
# Create a Sequence for the actual built GFP (mutant)
gfp_mutant_sequence = sbol2.Sequence(
    'gfp_mutant_sequence',
    elements='atggactacaaagacgatgacgacaaagctcttttagatccagtaatgaccttaca', # Notice the last three base pairs
    encoding=sbol2.SBOL_ENCODING_IUPAC
)
gfp_mutant_sequence.displayId = 'gfp_mutant_sequence'
gfp_mutant_sequence.version = '1.0.0'
doc.addSequence(gfp_mutant_sequence)

# Create the ComponentDefinition for the actual built GFP (mutant)
gfp_mutant = sbol2.ComponentDefinition('gfp_mutant')
gfp_mutant.displayId = 'gfp_mutant'
gfp_mutant.types =[ sbol2.BIOPAX_DNA] # Component type: DNA
gfp_mutant.roles=['http://identifiers.org/so/SO:0000316'] # Role: coding sequence (CDS)
gfp_mutant.version = '1.0.0'
gfp_mutant.sequences = [gfp_mutant_sequence] # Link the sequence to the ComponentDefinition
doc.addComponentDefinition(gfp_mutant)

### Create the Implementation Object
Now, we will create the `Implementation` object. We will link it to:

1.  The `gfp_design` using `wasDerivedFrom` (representing the original intent).
2.  The `gfp_mutant` using `built` (representing what was actually constructed and verified).

In [5]:
gfp_implementation = sbol2.Implementation('gfp_build_instance')
gfp_implementation.displayId = 'gfp_build_instance'
gfp_implementation.version = '1.0.0'

# Link to the original design using wasDerivedFrom
gfp_implementation.wasDerivedFrom = gfp_design.identity

# Link to the actual built construct using built
gfp_implementation.built = gfp_mutant.identity

doc.addImplementation(gfp_implementation)

### Validate and Save the Document
Finally, we'll validate the SBOL document to ensure it's well-formed and then save it to an XML file.

In [6]:
report = doc.validate()

if report == 'Valid.':
    doc.write('Implementation_example.xml')
else:
    print("Validation Errors:")
    print(report)
